In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
file = 'new-2017-2019cluster'
df = pd.read_csv('/Shared/taxi/{}.csv'.format(file),usecols=['Time', 'Week',
             'Trip_Miles','Trip_Seconds','mph','Pickup_Community_Area','Dropoff_Community_Area',
            'Tips', 'Trip_Total','pickup_id', 'Cluster_id'])
#usecols=['foo', 'bar']

In [ ]:
profit = df1.eval('(Trip_Total-(Trip_Miles*0.1255))/Trip_Miles')
df1['Profits']=profit

In [ ]:
# 是否給小費和額外費用
df1['Tips_yn']=(df1['Tips']/df1['Tips']).replace(np.nan,0)

In [ ]:
# 是否塞車(車速<10mph)
a_ = df1[df1['mph']<10]
a_['traffic_yn']=1
b_ = df1[df1['mph']>=10]
b_['traffic_yn']=0
df2=a_.append(b_)

In [ ]:
# 是否塞車(車速<10mph)
a_ = df1[df1['mph']<10]
a_['traffic_yn']=1
b_ = df1[df1['mph']>=10]
b_['traffic_yn']=0
df2=a_.append(b_)

In [ ]:
# 是否載出芝加哥
e_ = df3[df3['Dropoff_Community_Area']<0]
e_['outside_yn']=1
f_ = df3[df3['Dropoff_Community_Area']>=0]
f_['outside_yn']=0
df=e_.append(f_)

# GROUP

In [ ]:
group = df.groupby(["Cluster_id","Week","Time"])

In [ ]:
#med
med = group['Trip_Total','Trip_Miles','Trip_Seconds','Profits','mph'].median()
# 機率
mean = group['Tips_yn','long_yn','traffic_yn','outside_yn'].mean()
# 眾數
mode = group['Pickup_Community_Area']
y = mode.agg(lambda x: np.mean(x.mode()))
mode = group['Dropoff_Community_Area']
z = mode.agg(lambda x: np.mean(x.mode()))
# 載客機率
count = (group['Tips'].count()/156)
pickup_count = count.rename('pickup_count')
#pickup_p = (count/mean['picup_allCluster'])
#z = pickup_p.rename('pickup_p')

In [ ]:
# merge all dataframe
bb = pd.merge(pickup_count,med,left_index=True,right_index=True)
#bb = pd.merge(pickup_p,med,left_index=True,right_index=True)
dd = pd.merge(bb,mean,left_index=True,right_index=True)
cc=pd.merge(dd,y,left_index=True,right_index=True)
result=pd.merge(cc,z,left_index=True,right_index=True)
#dff1 = pd.merge(dff,z,left_index=True,right_index=True)
#dff1=dff.drop(['picup_allCluster_y'], axis=1)
#result=dff1.rename(columns={"picup_allCluster_x": "pickup_pro"})

In [ ]:
# save
result.to_csv('/Shared/taxi/final_description_2017-2019.csv')

# LEVEL

In [ ]:
#df = pd.read_csv('/Shared/taxi/final_description_2017-2019.csv')

In [ ]:
df = result.unsteak()

In [ ]:
# LABEL Trip_Total LEVEL
def level(x):
    if x <= 7.75:
        return 'Low'
    elif x > 7.75 and x <= 17.25:
        return 'Medium'
    else:
        return 'High'
df['Trip_Total_level']=df['Trip_Total'].apply(lambda x:level(x))

In [ ]:
# LABEL Profits LEVEL
def level(x):
    if x <= 3.89:
        return 'Low'
    elif x > 3.89 and x <= 8.62:
        return 'Medium'
    else:
        return 'High'
df['Profits_level']=df['Profits'].apply(lambda x:level(x))

In [ ]:
# LABEL Trips LEVEL
# def level(x):
#     if x <= 0.038:
#         return 'Low'
#     elif x > 0.038 and x <= 0.378:
#         return 'Medium'
#     else:
#         return 'High'
# df['Tips_yn_level']=df['Tips_yn'].apply(lambda x:level(x))

In [ ]:
# Turn into %
df[['Tips_yn','long_yn','traffic_yn','outside_yn']]=df[['Tips_yn','long_yn','traffic_yn','outside_yn']]*100

In [ ]:
# save
df.to_csv('/Shared/taxi/description17-19_0114.csv')